# NLP Analysis

# Imports

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Models 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from xgboost import XGBClassifier

# Data Processing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


np.random.seed(1)

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.feature_extraction.text import TfidfVectorizer

from collections import Counter

import torch
from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

# Functions

In [46]:
def read_dataframes(PATH, filepaths):

  dataframes = []

  for file in filepaths:
    dataframes.append(pd.read_csv(PATH + file +'.csv'))
  
  return dataframes

def check_class_imbalance(df):
  y = df.values[:,-1]
  counter = Counter(y)
  for k,v in counter.items():
    per = v/len(y) * 100
    print("Class=%s, Count=%d, Percentage=%.3f%%" % (k,v, per))

def sentiment_score(review, tokenizer, model):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) +1

def get_selected_models(names):
  """
  Returns selected models for ML processing

  Args:
      names (_type_):List

  Returns:
      List of models
  """
  models = {
    "LDA": LinearDiscriminantAnalysis(),
    "GPC": GaussianProcessClassifier(),
    "GNB": GaussianNB(),
    "SVC": SVC(),
    "LR":LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(),
    "DTC": DecisionTreeClassifier(),
    "GBC":GradientBoostingClassifier(),
    "RFC":RandomForestClassifier(),
    "XGB": XGBClassifier()
  }
  
  return [models[model] for model in names]
def evaluate_model(X, y, model):
  # define evaluation procedure
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  
  metric = make_scorer(accuracy_score)
  # evaluate model
  scores = cross_val_score(model, X, y, scoring=metric, cv=cv, n_jobs=-1)
  return scores
def testing_selected_models(names:list, models:list, X:pd.DataFrame, y:pd.Series):
    """
    Runs multiple subsets on folds of data

    Args:
        names (list): _description_
        models (list): _description_
    """
    
    for i in range(len(models)):
        model = models[i]
        # Evaluate the model
        scores = evaluate_model(X, y, model)
        # summarize and store
        print('>%s %.3f (%.3f)' % (names[i], np.mean(scores), np.std(scores)))

# Read in Dataframes

In [6]:
PATH = "/Users/maukanmir/Downloads/archive/"

filepaths = ['Education', 'Sports', 'Finance', 'Politics']

In [9]:
edu_df, sport_df, finance_df, politics_df = read_dataframes(PATH, filepaths)

# EDA Analysis

In [11]:

data_frames = [edu_df, sport_df, finance_df, politics_df ]

for df in data_frames:
  print(f"Number of NA values: {df.isna().sum()}")
  print(f"Number of Duplicated values: {df.duplicated().sum()}")
  print(df.shape)

Number of NA values: Text     0
Label    0
dtype: int64
Number of Duplicated values: 0
(52, 2)
Number of NA values: Text     0
Label    0
dtype: int64
Number of Duplicated values: 0
(56, 2)
Number of NA values: Text     0
Label    0
dtype: int64
Number of Duplicated values: 0
(48, 2)
Number of NA values: Text     0
Label    0
dtype: int64
Number of Duplicated values: 0
(53, 2)


# Check for class imbalances

In [15]:
for topic, df in zip(filepaths, data_frames):
  print("------------------------")
  print(f"Topic is {topic}")
  check_class_imbalance(df)

------------------------
Topic is Education
Class=positive, Count=26, Percentage=50.000%
Class=negative, Count=26, Percentage=50.000%
------------------------
Topic is Sports
Class=positive, Count=28, Percentage=50.000%
Class=negative, Count=28, Percentage=50.000%
------------------------
Topic is Finance
Class=positive, Count=34, Percentage=70.833%
Class=negative, Count=14, Percentage=29.167%
------------------------
Topic is Politics
Class=positive, Count=25, Percentage=47.170%
Class=negative, Count=28, Percentage=52.830%


In [28]:
MODEL = f'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [20]:
for df in data_frames:
  df["Label"] = df["Label"].apply(lambda x: 1 if x == 'negative' else 0)

In [29]:

for df in data_frames:
  df["BERT_score"] = df["Text"].apply(lambda review: sentiment_score(review, tokenizer, model))


In [32]:
tfidf = TfidfVectorizer(max_features=2000)

In [33]:
for idx, df in enumerate(data_frames):
  df["Topic"] = idx

In [35]:
final_df = pd.concat([edu_df, sport_df, finance_df, politics_df], axis=0, ignore_index=True)

In [37]:
final_df.shape

(209, 4)

# Separate Feature variables

In [38]:
X,y = final_df.drop("Label", axis=1), final_df['Label']

In [47]:
tfidf_matrix = tfidf.fit_transform(X['Text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# Concatenate with original data
X = pd.concat([X, tfidf_df], axis=1)